In [92]:
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import random
from datetime import datetime
import numpy as np
import re
import os
import geopandas as gpd
from datetime import datetime

font_path = r'C:/Users/user/NanumFontSetup_TTF_ALL/NanumGothic.ttf'
font_name = font_manager.FontProperties(fname=font_path, size=18).get_name()
rc('font',family=font_name)

In [93]:
resident_load_path = 'C:\\Data Analysis\\부평구 상권분석\\data\\전처리 파일\\주거인구\\기초구역별_주거인구_5181.shp'
worker_load_path = 'C:\\Data Analysis\\부평구 상권분석\\data\\전처리 파일\\직장인구\\기초구역별_직장인구_5181.shp'

worker_data = gpd.read_file(worker_load_path) # 직장인구
resident_data = gpd.read_file(resident_load_path) # 주거인구

In [94]:
basis_area = worker_data.drop_duplicates('BAS_ID',keep='first')
basis_area = basis_area.loc[:,['BAS_ID','geometry']].sort_values(by='BAS_ID').reset_index(drop=True)
basis_area

,BAS_ID,geometry
0,21300,"POLYGON ((172624.863 447152.276, 172670.940 44..."
1,21301,"POLYGON ((173295.105 447039.530, 173295.147 44..."
2,21302,"POLYGON ((173891.544 446881.024, 173891.543 44..."
3,21303,"POLYGON ((173668.839 446507.436, 173658.833 44..."
4,21304,"POLYGON ((173891.543 446881.019, 173891.500 44..."
...,...,...
156,21456,"POLYGON ((175340.378 442397.517, 175349.045 44..."
157,21457,"POLYGON ((178051.841 442436.153, 178041.770 44..."
158,21458,"POLYGON ((176592.215 442885.941, 176595.066 44..."
159,21459,"POLYGON ((176641.304 442645.152, 176647.527 44..."


# 1. 주거인구

In [117]:
resident_data.head()

,BAS_AR,BAS_ID,BAS_MGT_SN,CTP_KOR_NM,MVMN_DE,MVMN_RESN,NTFC_DE,OPERT_DE,SIG_CD,SIG_KOR_NM,...,POP_W_5054,POP_W_5559,POP_W_6064,POP_W_6500,pop_W,BD_MGT_SN,X_COORD,Y_COORD,DATA_LOAD_,geometry
0,2.648838,21459,2823700025,인천광역시,None,None,20121221,None,28237,부평구,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((176641.304 442645.152, 176647.527 44..."
1,0.156212,21384,2823700061,인천광역시,None,None,20121221,None,28237,부평구,...,12.81,10.19,13.34,23.10,129.11,2.820000e+23,930062.1230,1943822.453,202007.0,"POLYGON ((174332.882 443549.806, 174341.717 44..."
2,0.156212,21384,2823700061,인천광역시,None,None,20121221,None,28237,부평구,...,12.97,13.37,11.17,21.13,153.31,2.820000e+23,929909.9743,1943888.576,202007.0,"POLYGON ((174332.882 443549.806, 174341.717 44..."
3,0.156212,21384,2823700061,인천광역시,None,None,20121221,None,28237,부평구,...,14.03,18.35,15.11,21.51,154.75,2.820000e+23,929847.7744,1943761.796,202007.0,"POLYGON ((174332.882 443549.806, 174341.717 44..."
4,0.156212,21384,2823700061,인천광역시,None,None,20121221,None,28237,부평구,...,9.72,10.03,8.38,15.84,114.96,2.820000e+23,929858.0449,1943851.152,202007.0,"POLYGON ((174332.882 443549.806, 174341.717 44..."


In [118]:
display(resident_data.info())

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 19873 entries, 0 to 19872
Data columns (total 45 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   BAS_AR      19873 non-null  float64 
 1   BAS_ID      19873 non-null  object  
 2   BAS_MGT_SN  19873 non-null  object  
 3   CTP_KOR_NM  19873 non-null  object  
 4   MVMN_DE     0 non-null      object  
 5   MVMN_RESN   0 non-null      object  
 6   NTFC_DE     19873 non-null  object  
 7   OPERT_DE    0 non-null      object  
 8   SIG_CD      19873 non-null  object  
 9   SIG_KOR_NM  19873 non-null  object  
 10  STD_YYYY    19865 non-null  float64 
 11  BLOCK_CD    19865 non-null  float64 
 12  INVT_YYYY   19865 non-null  float64 
 13  HHLD_CNT    19865 non-null  float64 
 14  POP_M_0009  19865 non-null  float64 
 15  POP_M_1019  19865 non-null  float64 
 16  POP_M_2024  19865 non-null  float64 
 17  POP_M_2529  19865 non-null  float64 
 18  POP_M_3034  19865 non-null  float64 
 

None

In [119]:
pd.set_option("display.max_rows",10)

In [120]:
use_col = [1] + list(range(14,40))
res_use_data = resident_data.iloc[:,use_col]
res_use_data

,BAS_ID,POP_M_0009,POP_M_1019,POP_M_2024,POP_M_2529,POP_M_3034,POP_M_3539,POP_M_4044,POP_M_4549,POP_M_5054,...,POP_W_2529,POP_W_3034,POP_W_3539,POP_W_4044,POP_W_4549,POP_W_5054,POP_W_5559,POP_W_6064,POP_W_6500,pop_W
0,21459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21384,6.33,10.82,8.03,10.32,5.43,3.06,6.90,11.04,12.16,...,8.67,6.54,7.34,7.97,12.97,12.81,10.19,13.34,23.10,129.11
2,21384,20.04,16.57,11.71,11.99,5.18,13.94,13.69,12.38,14.78,...,9.03,8.44,12.20,13.55,14.88,12.97,13.37,11.17,21.13,153.31
3,21384,13.43,19.46,18.25,14.19,9.43,4.19,11.50,15.49,9.55,...,9.55,10.44,9.40,13.57,13.15,14.03,18.35,15.11,21.51,154.75
4,21384,15.03,12.42,8.78,8.99,3.88,10.46,10.26,9.28,11.08,...,6.77,6.33,9.15,10.16,11.16,9.72,10.03,8.38,15.84,114.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19868,21432,0.88,1.16,0.84,1.00,1.17,2.20,1.13,1.25,1.39,...,1.02,0.97,1.40,0.80,1.08,1.33,1.89,1.56,3.42,16.46
19869,21432,0.93,1.23,0.89,1.05,1.23,2.33,1.19,1.32,1.47,...,1.07,1.03,1.49,0.85,1.15,1.40,2.00,1.66,3.60,17.40
19870,21432,0.59,0.78,0.56,0.66,0.78,1.47,0.75,0.83,0.92,...,0.68,0.65,0.93,0.53,0.72,0.89,1.26,1.04,2.27,10.96
19871,21432,2.76,3.64,2.64,3.12,3.67,6.89,3.54,3.91,4.35,...,3.20,3.04,4.39,2.51,3.40,4.17,5.92,4.90,10.69,51.60


In [121]:
total_bas_id = set(res_use_data['BAS_ID'].unique())

In [122]:
res_use_data = res_use_data.dropna()
res_use_data.isnull().sum()

BAS_ID        0
POP_M_0009    0
POP_M_1019    0
POP_M_2024    0
POP_M_2529    0
             ..
POP_W_5054    0
POP_W_5559    0
POP_W_6064    0
POP_W_6500    0
pop_W         0
Length: 27, dtype: int64

In [123]:
temp_res = res_use_data.groupby('BAS_ID').sum().reset_index()
temp_res['total_resident_pop'] = temp_res['pop_M']+temp_res['pop_W']
temp_res = temp_res.sort_values(by='BAS_ID').reset_index(drop=True)
temp_res.head()

,BAS_ID,POP_M_0009,POP_M_1019,POP_M_2024,POP_M_2529,POP_M_3034,POP_M_3539,POP_M_4044,POP_M_4549,POP_M_5054,...,POP_W_3034,POP_W_3539,POP_W_4044,POP_W_4549,POP_W_5054,POP_W_5559,POP_W_6064,POP_W_6500,pop_W,total_resident_pop
0,21300,9.78,14.39,11.54,16.58,15.16,18.98,12.78,20.87,26.91,...,16.38,14.16,8.69,25.49,18.46,24.50,22.32,37.47,210.89,443.94
1,21301,20.31,51.67,24.57,43.67,23.91,28.45,33.60,41.59,48.40,...,26.19,25.81,24.55,42.09,48.96,59.90,43.89,78.94,475.72,963.88
2,21302,37.27,97.26,56.84,57.82,52.40,66.00,53.26,100.23,60.27,...,43.01,49.71,54.60,98.33,66.86,60.34,47.93,93.71,755.95,1531.01
3,21303,9.66,15.28,11.16,17.83,14.53,15.11,19.06,22.67,29.98,...,12.34,10.88,11.12,10.50,13.96,19.03,27.43,43.07,185.90,447.47
4,21304,28.24,40.74,26.78,32.70,40.32,44.52,45.69,75.43,80.01,...,30.96,30.97,33.04,51.79,53.12,53.13,57.98,107.55,567.67,1204.79


In [124]:
# 없어진 기초구역 확인
loss_bas_id = total_bas_id - set(temp_res['BAS_ID'].unique())
loss_bas_id

{'21327', '21339', '21343', '21372', '21375', '21386', '21459', '21460'}

In [125]:
sum_resident_by_bas_id = temp_res.iloc[:,[0,-1]]
sum_resident_by_bas_id

,BAS_ID,total_resident_pop
0,21300,443.94
1,21301,963.88
2,21302,1531.01
3,21303,447.47
4,21304,1204.79
...,...,...
148,21454,1533.27
149,21455,1076.49
150,21456,2.05
151,21457,3425.69


In [126]:
# 없어진 기초구역 데이터 삽입
for bas_id in list(loss_bas_id):
    temp_df = pd.DataFrame.from_dict([{'BAS_ID':bas_id,'total_resident_pop':0}])
    sum_resident_by_bas_id = pd.concat([sum_resident_by_bas_id,temp_df],axis=0)
    
# sort code 추가할것
len(sum_resident_by_bas_id)

161

In [127]:
resident_result = basis_area.merge(sum_resident_by_bas_id,how='inner',right_on='BAS_ID',left_on='BAS_ID')
resident_result.head()

,BAS_ID,geometry,total_resident_pop
0,21300,"POLYGON ((172624.863 447152.276, 172670.940 44...",443.94
1,21301,"POLYGON ((173295.105 447039.530, 173295.147 44...",963.88
2,21302,"POLYGON ((173891.544 446881.024, 173891.543 44...",1531.01
3,21303,"POLYGON ((173668.839 446507.436, 173658.833 44...",447.47
4,21304,"POLYGON ((173891.543 446881.019, 173891.500 44...",1204.79


# 2. 직장인구

In [128]:
worker_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 10776 entries, 0 to 10775
Data columns (total 42 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   BAS_AR      10776 non-null  float64 
 1   BAS_ID      10776 non-null  object  
 2   BAS_MGT_SN  10776 non-null  object  
 3   CTP_KOR_NM  10776 non-null  object  
 4   MVMN_DE     0 non-null      object  
 5   MVMN_RESN   0 non-null      object  
 6   NTFC_DE     10776 non-null  object  
 7   OPERT_DE    0 non-null      object  
 8   SIG_CD      10776 non-null  object  
 9   SIG_KOR_NM  10776 non-null  object  
 10  STD_YYYY    10774 non-null  float64 
 11  BLOCK_CD    10774 non-null  float64 
 12  INVT_YYYY   10774 non-null  float64 
 13  MAN_CNT_14  10774 non-null  float64 
 14  MAN_CNT_15  10774 non-null  float64 
 15  MAN_CNT_20  10774 non-null  float64 
 16  MAN_CNT_25  10774 non-null  float64 
 17  MAN_CNT_30  10774 non-null  float64 
 18  MAN_CNT_35  10774 non-null  float64 
 

In [129]:
# 필요 컬럼 추출
use_col = [1]+list(range(13,37))
worker_use_data = worker_data.iloc[:,use_col]
worker_use_data

,BAS_ID,MAN_CNT_14,MAN_CNT_15,MAN_CNT_20,MAN_CNT_25,MAN_CNT_30,MAN_CNT_35,MAN_CNT_40,MAN_CNT_45,MAN_CNT_50,...,WMAN_CNT_3,WMAN_CNT_4,WMAN_CNT_5,WMAN_CNT_6,WMAN_CNT_7,WMAN_CNT_8,WMAN_CNT_9,WMAN_CNT10,WMAN_CNT11,WMAN_CNT12
0,21459,0.0,0.30,4.68,1.09,0.47,0.34,0.25,0.26,0.26,...,0.13,0.11,0.08,0.06,0.04,0.03,0.01,0.01,0.00,0.00
1,21459,0.0,0.44,6.80,1.58,0.68,0.50,0.36,0.38,0.38,...,0.19,0.16,0.12,0.09,0.06,0.04,0.02,0.01,0.01,0.00
2,21384,0.0,0.00,0.25,0.62,0.99,0.86,1.11,0.86,0.37,...,0.86,2.96,5.06,6.91,2.59,3.21,1.36,0.62,0.12,0.12
3,21384,0.0,0.00,0.10,0.24,0.39,0.34,0.44,0.34,0.15,...,0.34,1.16,1.99,2.72,1.02,1.26,0.53,0.24,0.05,0.05
4,21384,0.0,0.00,0.24,0.60,0.96,0.84,1.08,0.84,0.36,...,0.84,2.88,4.92,6.72,2.52,3.12,1.32,0.60,0.12,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10771,21432,0.0,0.00,0.00,0.00,1.10,0.00,0.55,2.19,0.00,...,0.00,0.00,0.00,0.55,0.00,0.55,0.00,0.00,0.00,0.00
10772,21432,0.0,0.00,0.00,0.00,1.42,0.00,0.71,2.85,0.00,...,0.00,0.00,0.00,0.71,0.00,0.71,0.00,0.00,0.00,0.00
10773,21432,0.0,0.00,0.00,0.00,0.00,1.57,0.00,0.00,1.57,...,0.00,0.00,0.00,0.00,0.00,1.57,0.00,0.00,0.00,0.00
10774,21432,0.0,0.00,0.00,0.00,0.00,0.00,5.33,0.00,0.00,...,0.00,0.00,0.00,5.33,5.33,0.00,0.00,0.00,0.00,0.00


In [130]:
agg_worker_data = worker_use_data.groupby('BAS_ID').sum().reset_index()
agg_worker_data['total_worker_pop'] = agg_worker_data.iloc[:,1:25].agg('sum',axis=1)
agg_worker_data.iloc[:5,-1:]

,total_worker_pop
0,2968.24
1,1443.05
2,1702.21
3,70.93
4,614.88


In [131]:
worker_result = basis_area.merge(agg_worker_data,how='inner',right_on='BAS_ID',left_on='BAS_ID')
worker_result

,BAS_ID,geometry,MAN_CNT_14,MAN_CNT_15,MAN_CNT_20,MAN_CNT_25,MAN_CNT_30,MAN_CNT_35,MAN_CNT_40,MAN_CNT_45,...,WMAN_CNT_4,WMAN_CNT_5,WMAN_CNT_6,WMAN_CNT_7,WMAN_CNT_8,WMAN_CNT_9,WMAN_CNT10,WMAN_CNT11,WMAN_CNT12,total_worker_pop
0,21300,"POLYGON ((172624.863 447152.276, 172670.940 44...",1.58,1.58,49.86,267.59,279.16,271.01,348.97,345.74,...,99.68,132.94,136.33,184.43,219.40,116.34,44.91,13.30,8.34,2968.24
1,21301,"POLYGON ((173295.105 447039.530, 173295.147 44...",0.43,0.43,41.31,136.68,152.46,136.65,149.69,160.54,...,58.77,72.77,69.41,79.09,99.51,52.24,20.39,7.47,2.98,1443.05
2,21302,"POLYGON ((173891.544 446881.024, 173891.543 44...",0.00,0.00,71.57,176.07,197.65,139.12,142.77,151.82,...,123.64,89.58,67.58,83.18,73.34,58.12,21.54,0.00,0.95,1702.21
3,21303,"POLYGON ((173668.839 446507.436, 173658.833 44...",0.00,0.00,1.70,2.29,4.87,3.83,3.49,5.75,...,5.35,7.14,8.12,5.98,9.19,4.37,1.48,0.44,0.42,70.93
4,21304,"POLYGON ((173891.543 446881.019, 173891.500 44...",0.00,0.00,22.83,32.93,28.08,35.55,37.86,37.57,...,59.80,42.13,47.23,51.77,46.02,28.08,12.33,2.96,0.72,614.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,21456,"POLYGON ((175340.378 442397.517, 175349.045 44...",0.00,0.00,0.00,24.16,0.00,18.12,18.12,12.08,...,6.04,6.04,0.00,12.08,6.04,0.00,0.00,0.00,6.04,157.04
157,21457,"POLYGON ((178051.841 442436.153, 178041.770 44...",0.00,6.24,92.02,81.47,122.40,104.03,43.44,83.35,...,76.00,34.33,133.13,63.40,5.11,13.21,2.16,58.47,1.37,1109.41
158,21458,"POLYGON ((176592.215 442885.941, 176595.066 44...",0.00,0.04,1.18,19.23,10.60,22.10,25.68,26.57,...,11.06,34.19,32.65,33.52,20.30,23.31,2.99,0.41,0.20,308.86
159,21459,"POLYGON ((176641.304 442645.152, 176647.527 44...",0.00,0.74,11.48,2.67,1.15,0.84,0.61,0.64,...,0.27,0.20,0.15,0.10,0.07,0.03,0.02,0.01,0.00,20.53


# 3. 직장인구, 주거인구 비율 계산

In [132]:
ratio_pop = pd.DataFrame({'BAS_ID':basis_area['BAS_ID']
             ,'주거인구':resident_result['total_resident_pop']
             ,'직장인구':worker_result['total_worker_pop']
            ,'geometry':basis_area['geometry']})

ratio_pop['주거인구/직장인구'] = ratio_pop['주거인구'] / ratio_pop['직장인구']

# 4. 유동인구 대비 직장, 주거인구 비율

In [138]:
total_move_pop = pd.read_csv('../data/전처리 파일/유동인구/기초구역별 성연령 유동인구.csv',encoding='euc-kr')
ratio_pop['유동인구'] = total_move_pop['평균유동인구']
ratio_pop['(주거인구+직장인구) / 유동인구'] = (ratio_pop['주거인구'] + ratio_pop['직장인구']) / ratio_pop['유동인구']

In [139]:
ratio_pop.fillna(0,inplace=True)
ratio_pop.isnull().sum()

BAS_ID                0
주거인구                  0
직장인구                  0
geometry              0
주거인구/직장인구             0
유동인구                  0
(주거인구+직장인구) / 유동인구    0
dtype: int64

In [140]:
ratio_pop.iloc[:,-1].describe()

count    161.000000
mean       0.351128
std        0.258223
min        0.000000
25%        0.171511
50%        0.277065
75%        0.491993
max        1.304734
Name: (주거인구+직장인구) / 유동인구, dtype: float64

In [141]:
ratio_pop.to_csv('../data/전처리 파일/직장,상주인구 비율.csv',index=False,encoding='euc-kr')